In [1]:
import os
from dotenv import load_dotenv

from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

テキスト生成 - ストリーミング

In [2]:
_ = load_dotenv()
compartment_id = os.getenv("COMPARTMENT_ID")
service_endpoint = os.getenv("GENAI_ENDPOINT")

print("servce endpoint: ", service_endpoint)
print("compartment id: ", compartment_id)

servce endpoint:  https://inference.generativeai.us-chicago-1.oci.oraclecloud.com
compartment id:  ocid1.compartment.oc1..aaaaaaaanjtbllhqxcg67dq7em3vto2mvsbc6pbgk4pw6cx37afzk3tngmoa


In [13]:
chat = ChatOCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
    model_id="cohere.command-r-plus",
    is_stream=True,
    model_kwargs={
        "temperature": 0,
        "max_tokens": 500,
        "top_p": 0.75,
        "top_k": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
)

In [14]:
res = chat.stream(
    "OCHaCafeってなんですか？"
)

for chunk in res:
    print(chunk.content, end="")

OCHaCafe（オチャカフェ）は、オンライン茶道体験を提供する日本のスタートアップ企業です。茶道は日本の伝統的な文化の一つで、茶を点てて飲む儀式的な実践です。

OCHaCafe は、世界中の人々が日本の茶道の美しさと奥深さを発見できるようにすることを使命としています。彼らは、参加者が茶道の基本を学び、伝統的な茶室の雰囲気を体験できるインタラクティブで没入型のあるオンライン セッションを提供します。

OCHaCafe のセッションでは、参加者は茶道の起源、儀式、茶の準備と提供の方法など、茶道の歴史と文化について学びます。また、茶道の精神的な側面と、それが日本の文化と美学にどのように組み込まれているかについても説明します。

セッションは通常、ライブ ビデオ ストリーミングを介して行われ、参加者は茶道の専門家やインストラクターとやり取りし、質問し、リアルタイムで指導を受けることができます。セッション中には、茶道のさまざまな側面を体験できる実践的な演習も含まれます。

OCHaCafe は、人々が自宅や世界中のどこからでも日本の伝統文化に触れることができるようにすることで、茶道の芸術と実践をよりアクセスしやすくすることを目指しています。彼らのプラットフォームは、文化交流、学習、そしてユニークな日本の体験を求める人々のためのコミュニティの構築を目的としています。

OCHaCafe についての詳細は、彼らのウェブサイトまたはソーシャル メディア プラットフォームをご覧ください。そこでは、提供されるさまざまなセッション、スケジュール、参加方法に関する情報を見つけることができます。

エンベディング

In [15]:
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings

In [16]:
embeddings = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id
)

In [18]:
docs = [
    "「Oracle Cloud Hangout Cafe」(通称「おちゃかふぇ」/以降、OCHaCafe)は、日本オラクルが主催するコミュニティの1つです。定期的に、開発者・エンジニアに向けたクラウドネイティブな時代に身につけておくべきテクノロジーを深堀する勉強会を開催しています。",
    "日本オラクル株式会社（にほんオラクル、英: Oracle Corporation Japan）は、米国企業オラクルコーポレーション (Oracle Corporation) （1977年設立）が、1985年に日本で設立した法人である。"
]

res = embeddings.embed_documents(docs)

print("Dims: ", len(res[0]))

Dims:  1024
